## Load Model and Tokenizer

In [1]:
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    LlamaTokenizer
)
import torch
from datasets import load_dataset
def load_tw_llama_model(base_model):
    kwargs = {}
    kwargs["quantization_config"] = BitsAndBytesConfig(
        
    )

/home/guest/r12922050/miniconda3/envs/adl-hw3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_prompt(instruction):
    return (
    "Below is an instruction in mandarin that describes a translation task. "
    "Please carefully read the instruction and write a response that "
    "appropriately and accurately completes the task.\n\n"
    f"### Instruction: {instruction}\n\n"
    "### Response: "
    )

get_prompt("hi")

'Below is an instruction in mandarin that describes a translation task. Please carefully read the instruction and write a response that appropriately and accurately completes the task.\n\n### Instruction: hi\n\n### Response: '

In [3]:
data = load_dataset("json", data_files="data/train.json")
data

DatasetDict({
    train: Dataset({
        features: ['id', 'output', 'instruction'],
        num_rows: 10000
    })
})

In [4]:
data['train']

Dataset({
    features: ['id', 'output', 'instruction'],
    num_rows: 10000
})

In [1]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="data/train.json")
# random sample half of the dataset

/home/guest/r12922050/miniconda3/envs/adl-hw3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'DatasetDict' object has no attribute 'select'

In [5]:
from utils import get_prompt, add_prompt
map_data = data['train'].map(add_prompt, remove_columns=["instruction"])
map_data

Map: 100%|██████████| 10000/10000 [00:00<00:00, 27719.37 examples/s]


Dataset({
    features: ['id', 'output', 'input'],
    num_rows: 10000
})

In [8]:
map_data

Dataset({
    features: ['id', 'output', 'input'],
    num_rows: 10000
})

In [1]:
from datasets import load_dataset
data = load_dataset("json", data_files="data/train.json")['train']
data

/home/guest/r12922050/miniconda3/envs/adl-hw3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['id', 'output', 'instruction'],
    num_rows: 10000
})

In [2]:
# count average length of instructions
instruction_word_cnt = 0
output_word_cnt = 0
for i in range(len(data)):
    instruction_word_cnt += len(data[i]['instruction'])
    output_word_cnt += len(data[i]['output'])
instruction_word_cnt / len(data)

38.163

In [13]:
import json

train = json.load(open("data/train.json"))
train

[{'id': 'db63fb72-e211-4596-94a4-69617706f7ef',
  'instruction': '翻譯成文言文：\n雅裏惱怒地說： 從前在福山田獵時，你誣陷獵官，現在又說這種話。\n答案：',
  'output': '雅裏怒曰： 昔畋於福山，卿誣獵官，今復有此言。'},
 {'id': 'a48b0e8f-dc7a-4130-acc6-a91cc4a81bd1',
  'instruction': '沒過十天，鮑泉果然被拘捕。\n幫我把這句話翻譯成文言文',
  'output': '後未旬，果見囚執。'},
 {'id': 'f98882de-6962-46cf-a8f0-5d534eddda3a',
  'instruction': '辛未，命吳堅為左丞相兼樞密使，常楙參知政事。\n把這句話翻譯成現代文。',
  'output': '初五，命令吳堅為左承相兼樞密使，常增為參知政事。'},
 {'id': 'c491b5f1-fe54-4276-8dd2-cbc7a8d0f3c3',
  'instruction': '十八年，奚、契丹侵犯邊界，以皇上為河北道元帥，信安王為副，率禦史大夫李朝隱、京兆尹裴亻由先等八總管討伐他們。\n翻譯成文言文：',
  'output': '十八年，奚、契丹犯塞，以上為河北道元帥，信安王禕為副，帥禦史大夫李朝隱、京兆尹裴伷先等八總管兵以討之。'},
 {'id': 'bc8d68a3-cfe2-42ee-9d99-415380975642',
  'instruction': '正月，甲子朔，鼕至，太後享通天宮；赦天下，改元。\n把這句話翻譯成現代文。',
  'output': '聖曆元年正月，甲子朔，鼕至，太後在通天宮祭祀；大赦天下，更改年號。'},
 {'id': 'd5cd09fd-efaf-496c-be0b-b11e2e253d60',
  'instruction': '文言文翻譯：\n明日，趙用賢疏入。',
  'output': '答案：第二天，趙用賢的疏奏上。'},
 {'id': '97643fd2-d8c9-4cdc-8d6d-cad4724bd98e',
  'instruction': '我當時在三司，訪求太祖、仁宗的手書敕令沒有見到，然而人人能傳誦那些話，禁止私鹽

In [14]:
train[0]

{'id': 'db63fb72-e211-4596-94a4-69617706f7ef',
 'instruction': '翻譯成文言文：\n雅裏惱怒地說： 從前在福山田獵時，你誣陷獵官，現在又說這種話。\n答案：',
 'output': '雅裏怒曰： 昔畋於福山，卿誣獵官，今復有此言。'}

In [15]:
# for every train example, replace the instruction key name with input
for i in range(len(train)):
    train[i]['input'] = train[i].pop('instruction')
train

[{'id': 'db63fb72-e211-4596-94a4-69617706f7ef',
  'output': '雅裏怒曰： 昔畋於福山，卿誣獵官，今復有此言。',
  'input': '翻譯成文言文：\n雅裏惱怒地說： 從前在福山田獵時，你誣陷獵官，現在又說這種話。\n答案：'},
 {'id': 'a48b0e8f-dc7a-4130-acc6-a91cc4a81bd1',
  'output': '後未旬，果見囚執。',
  'input': '沒過十天，鮑泉果然被拘捕。\n幫我把這句話翻譯成文言文'},
 {'id': 'f98882de-6962-46cf-a8f0-5d534eddda3a',
  'output': '初五，命令吳堅為左承相兼樞密使，常增為參知政事。',
  'input': '辛未，命吳堅為左丞相兼樞密使，常楙參知政事。\n把這句話翻譯成現代文。'},
 {'id': 'c491b5f1-fe54-4276-8dd2-cbc7a8d0f3c3',
  'output': '十八年，奚、契丹犯塞，以上為河北道元帥，信安王禕為副，帥禦史大夫李朝隱、京兆尹裴伷先等八總管兵以討之。',
  'input': '十八年，奚、契丹侵犯邊界，以皇上為河北道元帥，信安王為副，率禦史大夫李朝隱、京兆尹裴亻由先等八總管討伐他們。\n翻譯成文言文：'},
 {'id': 'bc8d68a3-cfe2-42ee-9d99-415380975642',
  'output': '聖曆元年正月，甲子朔，鼕至，太後在通天宮祭祀；大赦天下，更改年號。',
  'input': '正月，甲子朔，鼕至，太後享通天宮；赦天下，改元。\n把這句話翻譯成現代文。'},
 {'id': 'd5cd09fd-efaf-496c-be0b-b11e2e253d60',
  'output': '答案：第二天，趙用賢的疏奏上。',
  'input': '文言文翻譯：\n明日，趙用賢疏入。'},
 {'id': '97643fd2-d8c9-4cdc-8d6d-cad4724bd98e',
  'output': '餘時在三司，求訪兩朝墨敕不獲，然人人能誦其言，議亦竟寢。',
  'input': '我當時在三司，訪求太祖、仁宗的手書敕令沒有見到，然而人人能傳誦那些話，禁止

In [16]:
# save the new train data
with open("data/train_modified.json", "w") as f:
    json.dump(train, f)

In [2]:
import json

train = json.load(open("data/train.json"))
train

[{'id': 'db63fb72-e211-4596-94a4-69617706f7ef',
  'instruction': '翻譯成文言文：\n雅裏惱怒地說： 從前在福山田獵時，你誣陷獵官，現在又說這種話。\n答案：',
  'output': '雅裏怒曰： 昔畋於福山，卿誣獵官，今復有此言。'},
 {'id': 'a48b0e8f-dc7a-4130-acc6-a91cc4a81bd1',
  'instruction': '沒過十天，鮑泉果然被拘捕。\n幫我把這句話翻譯成文言文',
  'output': '後未旬，果見囚執。'},
 {'id': 'f98882de-6962-46cf-a8f0-5d534eddda3a',
  'instruction': '辛未，命吳堅為左丞相兼樞密使，常楙參知政事。\n把這句話翻譯成現代文。',
  'output': '初五，命令吳堅為左承相兼樞密使，常增為參知政事。'},
 {'id': 'c491b5f1-fe54-4276-8dd2-cbc7a8d0f3c3',
  'instruction': '十八年，奚、契丹侵犯邊界，以皇上為河北道元帥，信安王為副，率禦史大夫李朝隱、京兆尹裴亻由先等八總管討伐他們。\n翻譯成文言文：',
  'output': '十八年，奚、契丹犯塞，以上為河北道元帥，信安王禕為副，帥禦史大夫李朝隱、京兆尹裴伷先等八總管兵以討之。'},
 {'id': 'bc8d68a3-cfe2-42ee-9d99-415380975642',
  'instruction': '正月，甲子朔，鼕至，太後享通天宮；赦天下，改元。\n把這句話翻譯成現代文。',
  'output': '聖曆元年正月，甲子朔，鼕至，太後在通天宮祭祀；大赦天下，更改年號。'},
 {'id': 'd5cd09fd-efaf-496c-be0b-b11e2e253d60',
  'instruction': '文言文翻譯：\n明日，趙用賢疏入。',
  'output': '答案：第二天，趙用賢的疏奏上。'},
 {'id': '97643fd2-d8c9-4cdc-8d6d-cad4724bd98e',
  'instruction': '我當時在三司，訪求太祖、仁宗的手書敕令沒有見到，然而人人能傳誦那些話，禁止私鹽

In [3]:
# random shuffle the train data and sample half of it
import random
random.shuffle(train)
train = train[:len(train)//2]
train

[{'id': 'dfe51d77-86a0-4c10-888c-a96afb6e9f27',
  'instruction': '馬肝石長時間用水銀保養，放在玉石做的匣子中，它的上麵用金泥封嚴。\n這句話在古代怎麼說：',
  'output': '長以水銀養，內於玉函中，金泥封其上。'},
 {'id': 'db2b5466-0d3d-4134-817d-341bab5b7582',
  'instruction': '九月二十五日，貶宇文融為汝州刺史。\n翻譯成古文：',
  'output': '九月壬子，貶宇文融為汝州刺史。'},
 {'id': '66913309-00de-4239-9d9d-d0298c17b672',
  'instruction': '當劉嚮死後，哀帝又派劉嚮的兒子侍中奉車都尉劉歆完成父親的事業，劉歆於是總結所有書籍而把《七略》上奏給皇帝。\n這句話在古代怎麼說：',
  'output': '會嚮卒，哀帝復使嚮子侍中奉車都尉歆卒父業。'},
 {'id': '023b5d69-50e0-4586-834d-d2385dc079af',
  'instruction': '丁巳，追冊為皇後。\n把這句話翻譯成現代文。',
  'output': '十八日，追命武氏為皇後。'},
 {'id': '28d7291b-126b-4197-b65b-c3eb45209358',
  'instruction': '翻譯成現代文：\n占曰： 有雲如眾風，是謂風師，法有大兵 。\n答案：',
  'output': '占候的入說： 有雲如大風，是謂風師，占法記載當有大的戰爭發動。'},
 {'id': 'd58afc13-74d4-4d8c-a168-9b8481799c77',
  'instruction': '曆代為邊患者，良以倏忽無常故也。\n把這句話翻譯成現代文。',
  'output': '曆代所以成為邊患的原因，在於他們齣沒無常的緣故。'},
 {'id': '20695882-938d-47e7-a13c-081990e3abd4',
  'instruction': '三年，劉誕占據廣陵城造反，派他的手下陸延稔帶著書信任命顧琛為徵南將軍，張牧為安南將軍，顧琛之子、前尚書郎顧寶素為諮議參軍，寶素之弟前司空參軍寶先當從事中郎

In [4]:
# save the new train data
with open("data/train_rand_half.json", "w") as f:
    json.dump(train, f, ensure_ascii=False)